# HYPERPARAMETERS OPTIMIZATION

## OBJECTIVE

The main objective of this notebook is finding the optimal parameters for the RNN Model.

Among the different hyperparameters the following values are considered:
    
    - Learning rate
    - Momentum
    - Distance
    - Batch_size
    - Initializer
    - Optimizer
    - Hidden Layers
    - Hidden Neurons

Two techniques will be used to optimize those values:

    -Grid Search
    -Random Search



In [1]:
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd
import keras
import keras.backend as Kback
from keras import layers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, SimpleRNN
from keras.layers.core import Activation
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

model_path = 'RNN_Hyp_1.h5'

# RNN_regression_PCA : First model with the highest accuracy 
# RNN_regression_PCA_1: Increasing the variability explained by PCA from 0.95 to 0.98
# RNN_regression_PCA_2: Reducing the sequence from 50 to 30
# RNN_regression_PCA_3: Increasing the sequence from 50 to 60

import random
random.seed(123)


Using TensorFlow backend.


In [2]:
from math import e

In [3]:
#######
#Data ingestion & processing
######

train_df = pd.read_csv("C:/Users/eduardo.tadeo/Documents/Master Thesis/Datasets/CMAPSSData/train_FD001.txt", delimiter = ',')
test_df  = pd.read_csv("C:/Users/eduardo.tadeo/Documents/Master Thesis/Datasets/CMAPSSData/test_FD001.txt", delimiter = ',')
RUL_test = pd.read_csv("C:/Users/eduardo.tadeo/Documents/Master Thesis/Datasets/CMAPSSData/RUL_FD001.txt")

#Unuseful columns deleting 

train_df.drop(['T_EGT', 'SmFan', 'SmLPC', 'SmHPC'], axis = 1, inplace = True)

test_df.drop(['T_EGT', 'SmFan', 'SmLPC', 'SmHPC'], axis = 1, inplace = True)


###TRAIN DATA SET###

# Standarization for both sensor variables and operational conditions

train_df['Cycle_norm'] = train_df['Cycle']
cols_normalize = train_df.columns.difference(['Unit','Cycle','UL','RUL','UL_30','UL_50', 'UL_75'])
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_df[cols_normalize])
norm_train_df = pd.DataFrame(scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)

# Principal Component Analysis Dimensionality Reduction

pca = PCA(.95)

pca.fit(norm_train_df)
transformed_train = pca.transform(scaled_train)

df_transformed_train = pd.DataFrame(transformed_train, index = train_df.index)

join_df = train_df[train_df.columns.difference(cols_normalize)].join(df_transformed_train)
train_df = join_df.reindex()


# Data labeling - Remaining Useful LIfe (RUL) --> Time to failure

RUL = pd.DataFrame(train_df.groupby('Unit')['Cycle'].max()).reset_index()
RUL.columns = ['Unit','UL']
train_df = train_df.merge(RUL, on = ['Unit'], how = 'left')
train_df['RUL'] = train_df['UL'] - train_df['Cycle']


# Data labeling - UL_30 - UL_50 - UL_75 --> Labeling to predict if the turbine is going to fail on les than 30, 50 or 75 cycles

train_df['UL_30'] = np.where(train_df['RUL'] <= 30, 1, 0)
train_df['UL_50'] = np.where(train_df['RUL'] <= 50, 1, 0)
train_df['UL_75'] = np.where(train_df['RUL'] <= 75, 1, 0)


In [4]:
train_df.head()

,Cycle,Unit,0,1,2,3,4,5,6,7,8,9,10,11,12,UL,RUL,UL_30,UL_50,UL_75
0,1,1,-3.240845,-0.559371,-1.176731,0.489354,-0.674640,0.769576,-0.669039,0.631733,-0.421074,0.039968,0.643934,0.289824,0.222595,192,191,0,0,0
1,2,1,-2.669143,-0.940659,-0.137753,1.167902,-0.706549,0.927689,-0.789666,0.287941,-0.121802,-0.306724,0.658158,0.108694,-0.231444,192,190,0,0,0
2,3,1,-3.260073,-0.665652,-0.528376,-2.116760,0.358836,0.752253,-0.429145,-1.129094,-0.590819,0.269416,0.098841,-0.111126,0.240092,192,189,0,0,0
3,4,1,-3.655610,-0.931102,0.253664,0.143214,-0.343321,0.521816,0.534238,-0.070660,0.000428,-0.105096,-1.125582,0.385607,0.633467,192,188,0,0,0
4,5,1,-2.716606,-0.510696,-1.027479,-0.294257,-0.363062,0.651352,0.902064,0.330613,-0.084591,-0.475373,0.228016,-0.122698,0.070388,192,187,0,0,0


In [5]:
###TEST DATA SET###


#Normalize data
test_df['Cycle_norm'] = test_df['Cycle']
norm_test_df = pd.DataFrame(scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)

print(norm_train_df.shape,norm_test_df.shape)

#Principal Component Analysis Dimensionality Reduction

transformed_test = pca.transform(norm_test_df)

df_transformed_test = pd.DataFrame(transformed_test, index = test_df.index)

join_df = test_df[test_df.columns.difference(cols_normalize)].join(df_transformed_test)
test_df = join_df.reindex()

# We use the ground truth dataset to generate labels for the test data.
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('Unit')['Cycle'].max()).reset_index()
rul.columns = ['Unit', 'max']
RUL_test.columns = ['more']
RUL_test['Unit'] = RUL_test.index + 1
RUL_test['max'] = rul['max'] + RUL_test['more']
RUL_test.drop('more', axis=1, inplace=True)

# generate RUL for test data
test_df = test_df.merge(RUL_test, on=['Unit'], how='left')
test_df['RUL'] = test_df['max'] - test_df['Cycle']
#test_df.drop('max', axis=1, inplace=True)

#Data labeling - UL_30 - UL_50 - UL_75 --> Labeling to predict if the turbine is going to fail on les than 30, 50 or 75 cycles

test_df['UL_30'] = np.where(test_df['RUL'] <= 30, 1, 0)
test_df['UL_50'] = np.where(test_df['RUL'] <= 50, 1, 0)
test_df['UL_75'] = np.where(test_df['RUL'] <= 75, 1, 0)


(20631, 25) (13096, 25)


In [6]:
#####
#Data formatting#
####

# Data will be input into the RNN in windows with a certain size
sequence_length = 50

# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

sequence_cols = list(train_df.columns[2:-5].values)
#sequence_cols.extend(["Cycle_norm"])

#val=list(gen_sequence(train_df[train_df['Unit']==1], sequence_length, sequence_cols))

seq_gen = (list(gen_sequence(train_df[train_df['Unit']==id], sequence_length, sequence_cols)) 
           for id in train_df['Unit'].unique())


# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]] 
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['Unit']==id], sequence_length, ['RUL']) 
             for id in train_df['Unit'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape


(15631, 50, 13)


(15631, 1)

In [7]:
def r2_keras(y_true, y_pred):
    ##Coefficient of Determination##

    SS_res = Kback.sum(Kback.square(y_true - y_pred))
    SS_tot = Kback.sum(Kback.square( y_true - Kback.mean(y_true)))
    return (1 - SS_res/(SS_tot + Kback.epsilon()))

def Computed_Score(y_true, y_pred):
    ##Computed score used in the challenge

    a1 = 10
    a2 = 13
    score = 0
    d = y_pred - y_true

    for i in d: 
        if i<0:
            score += (e**(-i/a1) - 1)
        else : 
            score += (e**(i/a2) - 1)
    
    return score


In [8]:
nb_features = seq_array.shape[2]


In [9]:
def hyp_model(init_mode = 'uniform', optim = 'ADAM',hidd_layers = 1, hidd_neurons = 10, sequence_length = 50, nb_features =13 ):

    # Define the model

    model = Sequential()
    model.add(LSTM(
        input_shape = (sequence_length, nb_features),
        units = nb_features,
        return_sequences = True,
        kernel_initializer= init_mode))
    model.add(Dropout(0.1))
    for i in range(hidd_layers):
        model.add(LSTM(
            units = hidd_neurons,
            return_sequences = False,
            kernel_initializer=init_mode))
        model.add(Dropout(0.1))
    model.add(Dense(units = 1))
    model.add(Activation("linear"))
    
    # Compile the model

    model.compile(loss = 'mean_squared_error', optimizer = optim,
        metrics= ['mae', r2_keras])
    
    print(model.summary())

    return model


In [10]:
keras.optimizers.get('ADAMAX')

In [11]:
# Hyperparameters definition

batches = [50,100,200,500]
initializers = ['uniform', 'lecun_uniform'] # 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'
optimizers = ['adam', 'rmsprop', 'SGD','adamax', 'adadelta', 'adagrad']
hidden_layers = [1,2]
hidden_neurons = [7,8,9,10,11,12]



In [12]:
model_CV = KerasRegressor(build_fn=hyp_model, verbose = 1, epochs = 10, validation_split = 0.1)

# Grid Search for batches, initializer, optimizer, hidden layers and neurons.

param_grid = dict(init_mode = initializers, batch_size = batches)

grid = GridSearchCV(estimator = model_CV, param_grid=param_grid, cv = 3)

grid_result = grid.fit(seq_array, label_array, 
    callbacks = [keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience=10, verbose = 1, mode = 'min'), 
        keras.callbacks.ModelCheckpoint(model_path, monitor = 'val_loss', save_best_only = True, mode = 'min', verbose = 0)])



13013.1901 - val_mae: 92.2569 - val_r2_keras: -2.4122
Epoch 6/10
9378/9378 [==============================] - 3s 319us/step - loss: 10150.2629 - mae: 81.7767 - r2_keras: -1.9149 - val_loss: 12925.4786 - val_mae: 91.8049 - val_r2_keras: -2.3568
Epoch 7/10
9378/9378 [==============================] - 3s 325us/step - loss: 10078.2820 - mae: 81.3620 - r2_keras: -1.8944 - val_loss: 12855.2133 - val_mae: 91.4433 - val_r2_keras: -2.3130
Epoch 8/10
9378/9378 [==============================] - 3s 338us/step - loss: 10019.0832 - mae: 81.0261 - r2_keras: -1.8812 - val_loss: 12794.8634 - val_mae: 91.1339 - val_r2_keras: -2.2759
Epoch 9/10
9378/9378 [==============================] - 3s 357us/step - loss: 9966.7057 - mae: 80.7179 - r2_keras: -1.8657 - val_loss: 12739.7226 - val_mae: 90.8505 - val_r2_keras: -2.2426
Epoch 10/10
5211/5211 [==============================] - 1s 117us/step
Model: "sequential_20"
_________________________________________________________________
Layer (type)               

In [13]:
# print results
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for -6.844e+03 using {'batch_size': 50, 'init_mode': 'uniform'}
mean=-6.844e+03, std=1.034e+03 using {'batch_size': 50, 'init_mode': 'uniform'}
mean=-6.85e+03, std=995.1 using {'batch_size': 50, 'init_mode': 'lecun_uniform'}
mean=-8.062e+03, std=1.038e+03 using {'batch_size': 100, 'init_mode': 'uniform'}
mean=-8.081e+03, std=1.245e+03 using {'batch_size': 100, 'init_mode': 'lecun_uniform'}
mean=-8.943e+03, std=1.223e+03 using {'batch_size': 200, 'init_mode': 'uniform'}
mean=-8.933e+03, std=1.235e+03 using {'batch_size': 200, 'init_mode': 'lecun_uniform'}
mean=-9.508e+03, std=1.234e+03 using {'batch_size': 500, 'init_mode': 'uniform'}
mean=-9.469e+03, std=1.235e+03 using {'batch_size': 500, 'init_mode': 'lecun_uniform'}
